In [1]:
from xgboost import XGBClassifier
import pandas as pd


In [2]:
data = pd.read_csv("DATA/MinMax_Scaler_result.csv",encoding="cp949")

In [3]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','ONEHOT']]
act = data['ACTCD']

In [4]:
from sklearn.model_selection import train_test_split

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 22) (11736, 22) (27381,) (11736,)


In [5]:
'''
1. ENN
2. ADASYN
3. BORDERLINE SMOTE
4. Tomeklinks
5 SOTEENN
6.SMOTETOMEK
'''
from imblearn.over_sampling import BorderlineSMOTE,ADASYN
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
data_sets={}
enn = EditedNearestNeighbours()
ada = ADASYN(random_state=42)
smte = SMOTEENN(random_state=42)
smtt = SMOTETomek(random_state=42)
tk = TomekLinks()
sm = BorderlineSMOTE(random_state=42)
enn_under_train_x,enn_under_train_y = enn.fit_resample(train_x,train_y)
ada_over_train_x, ada_over_train_y = ada.fit_resample(train_x,train_y)
smte_combine_train_x,smte_combine_train_y = smte.fit_resample(train_x,train_y)
smtt_combine_train_x,smtt_combine_train_y = smtt.fit_resample(train_x,train_y)
tk_under_train_x,tk_under_train_y = tk.fit_resample(train_x,train_y)
sm_over_train_x, sm_over_train_y = sm.fit_resample(train_x,train_y)
data_sets['enn']=[enn_under_train_x,enn_under_train_y]
data_sets['ada']=[ada_over_train_x,ada_over_train_y]
data_sets['smte']=[smte_combine_train_x,smte_combine_train_y]
data_sets['smtt']=[smtt_combine_train_x,smtt_combine_train_y]
data_sets['tk']=[tk_under_train_x,tk_under_train_y]
data_sets['sm']=[sm_over_train_x,sm_over_train_y]
'''
over_train_x : 학습용 독립변수
over_train_y : 학습용 종속변수
'''
keys_list= data_sets.keys()

In [6]:
xgb = XGBClassifier(
    learning_rate =0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=-1,
    scale_pos_weight=1,
    seed=2019
)

xgb_param_gird = {
    'n_estimators' : [100,500],
    'learning_rate' : [0.01, 0.2],
    'max_depth':[4]
}

In [7]:
from sklearn.model_selection import GridSearchCV

In [8]:
xgb_gird = GridSearchCV(xgb,param_grid=xgb_param_gird, scoring="accuracy",n_jobs=3,verbose=1)
xgb_gird.fit(enn_under_train_x,enn_under_train_y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed:   30.1s finished
C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:19:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     enable_categorical=False, gamma=0,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=5, min_child_weight=1,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=1000, n_jobs=None, nthread=-1,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=1,
             